In [9]:
import pandas as pd
import sqlite3

name = 'Alejandra Camino'

conn = sqlite3.connect("normalized.db")
sql_statement = """ 
        
       WITH orders AS (
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        ct.Country,
        o.OrderDate,
        LAG(o.OrderDate) OVER (PARTITION BY c.CustomerID ORDER BY o.OrderDate) AS PreviousOrderDate
    FROM OrderDetail o
    JOIN Product p ON o.ProductID = p.ProductID
    JOIN Customer c ON o.CustomerID = c.CustomerID
    JOIN Country ct ON c.CountryID = ct.CountryID
    JOIN Region r ON r.RegionID = ct.RegionID
)

SELECT CustomerID,FirstName, LastName, Country, OrderDate, PreviousOrderDate,
    MAX(JULIANDAY(OrderDate) - JULIANDAY(PreviousOrderDate)) AS MaxDaysWithoutOrder
FROM orders
GROUP BY CustomerID
ORDER BY MaxDaysWithoutOrder DESC

     """
df = pd.read_sql_query(sql_statement, conn)
display(df)
conn.close()


,CustomerID,FirstName,LastName,Country,OrderDate,PreviousOrderDate,MaxDaysWithoutOrder
0,70,Patricio,Simpson,Argentina,2015-03-20,2015-01-01,78.0
1,26,Frederique,Citeaux,France,2014-05-02,2014-02-23,68.0
2,54,Manuel,Pereira,Venezuela,2015-04-24,2015-02-18,65.0
3,8,Antonio,Moreno,Mexico,2015-06-16,2015-04-14,63.0
4,23,Felipe,Izquierdo,Venezuela,2014-12-16,2014-10-16,61.0
...,...,...,...,...,...,...,...
86,51,Liu,Wong,USA,2014-11-05,2014-10-06,30.0
87,64,Michael,Holz,Switzerland,2015-06-16,2015-05-18,29.0
88,41,Jean,Fresniere,Canada,2016-01-24,2015-12-27,28.0
89,28,Giovanni,Rovelli,Italy,2012-12-02,2012-11-04,28.0


In [10]:
conn.close()